This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify non-hyperelliptic, non-trigonal candidates for the curve $C'$ in a purely geometric extensions $F'/F$ of relative class number 1 in which $g_F = 1$, $g_{F'} = 5$. Allow 3 hours for completion.

In [18]:
import itertools
from collections import defaultdict

Construct the points of $\mathbb{P}^4$ over $\mathbb{F}_2$, $\mathbb{F}_4$, $\mathbb{F}_8$, and $\mathbb{F}_{16}$.

In [2]:
F = GF(2)
P.<x0,x1,x2,x3,x4> = F[]

In [3]:
S = [vector(F,t) for t in itertools.product(F, repeat=5) if not all(i==0 for i in t)]
for v in S:
    v.set_immutable()
len(S)

31

In [37]:
F4 = GF(4)
S4 = [vector(F4,t) for t in itertools.product(F4, repeat=5) if not all(i==0 for i in t) and t[min(i for i in range(5) if t[i])] == 1]
for v in S4:
    v.set_immutable()
len(S4)

341

In [85]:
F8 = GF(8)
S8 = [vector(F8,t) for t in itertools.product(F8, repeat=5) if not all(i==0 for i in t) and t[min(i for i in range(5) if t[i])] == 1]
for v in S8:
    v.set_immutable()
len(S8)

4681

In [125]:
F16 = GF(16)
S16 = [vector(F16,t) for t in itertools.product(F16, repeat=5) if not all(i==0 for i in t) and t[min(i for i in range(5) if t[i])] == 1]
for v in S16:
    v.set_immutable()
len(S16)

69905

Construct equivalence class of nondegenerate quadratic forms.

In [4]:
monos2 = [prod(t) for t in itertools.combinations_with_replacement(P.gens(), 2)]
len(monos2)

15

In [5]:
coords2 = {x: vector(mu(*x) for mu in monos2) for x in S}

In [150]:
quads = {1: [], 2: [], 3: [], 4: []}
for t in itertools.product(F, repeat=15):
    v = vector(t)
    v.set_immutable()
    gen = sum(t[i]*monos2[i] for i in range(15))
    if gen == 0: 
        continue
    fac = gen.factor()
    if len(fac) > 1 or fac[0][1] > 1:
        continue
    pts = [x for x in S if gen(*x) == 0]
    if len(pts) == 19:
        quads[2].append(v)
    elif len(pts) == 11:
        quads[3].append(v)
    elif len(pts) == 15:
        J = P.ideal([gen]) + gen.jacobian_ideal()
        if J.dimension() == 0:
            quads[4].append(v)
        else:
            quads[1].append(v)

In [151]:
[len(quads[i]) for i in range(1,5)]

[4340, 8680, 5208, 13888]

In [172]:
quads_lookup = {v: i for i in quads for v in quads[i]}
for t in itertools.product(F, repeat=15):
    v = vector(t)
    v.set_immutable()
    if v not in quads_lookup:
        quads_lookup[v] = 0

For one representative of each quadratic form, find the $\mathbb{F}_2$-points of the corresponding hypersurface.

In [159]:
d = {}
for i in range(1, 5):
    v = quads[i][0]
    gen1 = sum(v[i]*monos2[i] for i in range(15))
    d[v] = [x for x in S if gen1(*x) == 0]

Identify our target point counts.

In [160]:
targets = [(0,0,9,12,50),
           (0,0,12,16,60),
           (0,2,6,18,30),
           (0,2,3,42,30),
           (0,4,0,16,20),
           (0,4,0,28,25),
           (0,6,0,18,25),
           (1,1,1,25,1)]
targets2 = [tuple(t[:2]) for t in targets]
targets3 = [tuple(t[:3]) for t in targets]
targets4 = [tuple(t[:4]) for t in targets]

Loop through triples of quadrics $X_1, X_2, X_3$. Notes:

- All of the quadrics in the pencil must be nondegenerate.
- We may assume that $X_1$ is the first listed representative in its equivalence classes, and that $X_2, X_3$ are in later equivalence classes.
- Given $X_1, X_2$, we solve a linear equation to find candidates for $X_3$ so that $(X_1 \cap X_2 \cap X_3)(\mathbb{F}_2)$ consists of at most one point (which are free to specify).
- Once we have a candidate with the right $\mathbb{F}_2$-point, we then check the counts over $\mathbb{F}_4$, $\mathbb{F}_8$, $\mathbb{F}_{16}$ in turn. The code is set up so that we compute the points of $X_1 \cap X_2$ at most once for each choice of $X_1, X_2$ (and only if it is needed).

In [194]:
d2 = defaultdict(list)
for i1 in range(1, 5):
    v1 = quads[i1][0]
    gen1 = sum(v1[i]*monos2[i] for i in range(15))
    print(gen1)
    pts21 = [x for x in S4 if gen1(*x) == 0]
    pts31 = [x for x in S8 if gen1(*x) == 0]
    pts41 = [x for x in S16 if gen1(*x) == 0]
    b = []
    for v in identity_matrix(F, 15).rows():
        if Matrix([v1,v] + b).rank() > Matrix([v1] + b).rank():
            b.append(v)
    for v2 in Matrix(b).row_space():
#        if all(v2[i] == 0 for i in range(8)):
#            print(v2)
        v2.set_immutable()
        i2 = quads_lookup[v2]
        v = v1+v2
        v.set_immutable()
        if i2 < i1 or quads_lookup[v] < i1:
            continue
        gen2 = sum(v2[i]*monos2[i] for i in range(15))
        pts = [x for x in d[v1] if v2*coords2[x] == 0]
        perp = Matrix([coords2[x] for x in pts])
        target = vector(F(1) for x in pts)
        try:
            v3 = perp.solve_right(target)
        except ValueError:
            continue
        W = perp.right_kernel()
        b = []
        for v in W.basis():
            if Matrix([v1,v2,v] + b).rank() > Matrix([v1,v2] + b).rank():
                b.append(v)
        W = W.subspace(b)
        pts2 = 0
        pts3 = 0
        pts4 = 0
        for w in W:
            tmp = [v3+w+v1*j1+v2*j2 for j1 in F for j2 in F]
            for v in tmp:
                v.set_immutable()
            if any(quads_lookup[v] < i2 for v in tmp):
                continue
            if pts2 == 0:
                pts2 = [x for x in pts21 if gen2(*x) == 0]
            gen3 = sum((v3+w)[i]*monos2[i] for i in range(15))
            s = 0
            s2 = sum(1 for x in pts2 if gen3(*x) == 0)
            if (s,s2) in targets2:
                if pts3 == 0:
                    pts3 = [x for x in pts31 if gen2(*x) == 0]
                s3 = sum(1 for x in pts3 if gen3(*x) == 0)
                if (s,s2,s3) in targets3:
                    if pts4 == 0:
                        pts4 = [x for x in pts41 if gen2(*x) == 0]
                    s4 = sum(1 for x in pts4 if gen3(*x) == 0)
                    if (s,s2,s3,s4) in targets4 and M not in d2[(s,s2,s3,s4)]:
                        if len(d2[(s,s2,s3,s4)]) == 0:
                            print(s,s2,s3,s4)
                        M = Matrix([v1,v2,v3+w]).echelon_form()
                        d2[(s,s2,s3,s4)].append(M)

x3^2 + x2*x4
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0)
0 4 0 16
0 0 9 12
0 0 12 16
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,

(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1)
x2^2 + x1*x3 + x0*x4
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 

In [195]:
d3 = {}
for s in targets4:
    if s in d2:
        d3[s] = [[sum(v[i]*monos2[i] for i in range(15)) for v in M.rows()] for M in d2[s]]
{s: len(d3[s]) for s in d3}

{(0, 0, 9, 12): 1183, (0, 0, 12, 16): 56, (0, 4, 0, 16): 1177}

Use Magma to compute the full zeta functions of the resulting curves, and compare these to our targets.

In [196]:
counts = defaultdict(list)
proj = magma.ProjectiveSpace(P)
for s in d3:
    for gens in d3[s]:
        X = proj.Scheme(gens)
        if X.Dimension() != 1 or str(X.IsIrreducible()) == "false" or str(X.IsNonsingular()) == "false":
            continue
        C = X.Curve()
        ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 6))
        if ct in targets and not counts[ct]:
            print(ct)
        counts[ct].append(gens)
        if all(t in counts for t in targets if t[:3] == s):
            break
    for t in targets:
        if t[:3] == s and t not in counts:
             print("Counts not found: {}".format(t))

(0, 0, 9, 12, 50)
(0, 0, 12, 16, 60)
(0, 4, 0, 16, 20)


In [197]:
{t: len(counts[t]) for t in targets}

{(0, 0, 9, 12, 50): 1,
 (0, 0, 12, 16, 60): 1,
 (0, 2, 6, 18, 30): 0,
 (0, 2, 3, 42, 30): 0,
 (0, 4, 0, 16, 20): 1,
 (0, 4, 0, 28, 25): 0,
 (0, 6, 0, 18, 25): 0,
 (1, 1, 1, 25, 1): 0}

In [ ]:
print("All tests completed!")